In [1]:
from argparse import ArgumentParser

import numpy as np
import pandas as pd
from Bio import SeqIO
from typeguard import typechecked

In [2]:
STOP_CODONS = ("TAG", "TAA", "TGA")
START_CODONS = "ATG"


In [14]:
input_file_path = '../../data/protein/transcripts.ORF_prob.tsv'
orf_input_seq_path= '../../data/protein/transcripts.ORF_seqs.fa'
output_path='../../data/protein/transcripts.ORF_remaining.tsv'
first_cutoff=0.725
second_cutoff=0.725

In [15]:
full_orf_ids = []
for record in SeqIO.parse(orf_input_seq_path, "fasta"):
    if str(record.seq).upper().endswith(STOP_CODONS) and str(
        record.seq
    ).upper().startswith(START_CODONS):
        full_orf_ids.append(record.id)

In [16]:
full_orf_ids[:5]

['TRANSCRIPT_193634_ORF_1',
 'TRANSCRIPT_193634_ORF_2',
 'TRANSCRIPT_203700_ORF_1',
 'TRANSCRIPT_203700_ORF_2',
 'TRANSCRIPT_203700_ORF_3']

In [17]:
df.head()

,ID,mRNA,ORF_strand,ORF_frame,ORF_start,ORF_end,ORF,Fickett,Hexamer,Coding_prob,transcript_id
0,TRANSCRIPT_193634_ORF_1,610,+,3,279,512,234,0.6998,0.025784,0.064453,TRANSCRIPT
1,TRANSCRIPT_193634_ORF_2,610,+,1,97,270,174,0.8367,0.450741,0.426588,TRANSCRIPT
2,TRANSCRIPT_203700_ORF_1,1475,+,2,884,1135,252,1.0809,0.490912,0.834105,TRANSCRIPT
3,TRANSCRIPT_203700_ORF_2,1475,+,1,1144,1377,234,0.6998,0.025784,0.061652,TRANSCRIPT
4,TRANSCRIPT_203700_ORF_3,1475,+,2,443,607,165,1.2931,-0.113683,0.091480,TRANSCRIPT


In [18]:
df = pd.read_csv(f"{input_file_path}", sep="\t")
print(len(df.index))
df = df.loc[df.ID.isin(full_orf_ids)].copy(deep=True)
print(len(df.index))


345240
291857


In [19]:
df["transcript_id"] = df.ID.str.rsplit("_", n=2).str[0]

df_first_cutoff = df.loc[df["Coding_prob"] > first_cutoff]
print(len(df.index))

df_second_cutoff = df.loc[df["Coding_prob"] > second_cutoff]
print(len(df.index))


291857
291857


In [20]:
df.head()

,ID,mRNA,ORF_strand,ORF_frame,ORF_start,ORF_end,ORF,Fickett,Hexamer,Coding_prob,transcript_id
0,TRANSCRIPT_193634_ORF_1,610,+,3,279,512,234,0.6998,0.025784,0.064453,TRANSCRIPT_193634
1,TRANSCRIPT_193634_ORF_2,610,+,1,97,270,174,0.8367,0.450741,0.426588,TRANSCRIPT_193634
2,TRANSCRIPT_203700_ORF_1,1475,+,2,884,1135,252,1.0809,0.490912,0.834105,TRANSCRIPT_203700
3,TRANSCRIPT_203700_ORF_2,1475,+,1,1144,1377,234,0.6998,0.025784,0.061652,TRANSCRIPT_203700
4,TRANSCRIPT_203700_ORF_3,1475,+,2,443,607,165,1.2931,-0.113683,0.091480,TRANSCRIPT_203700


In [12]:
temp = pd.concat(
    [
        df_first_cutoff.sort_values(["ORF", "Coding_prob"], ascending=False)
        .groupby("transcript_id")
        .head(1),
        df_second_cutoff.sort_values(["ORF", "Coding_prob"], ascending=False)
        .groupby("transcript_id")
        .head(1),
        df.sort_values(["ORF", "Coding_prob"], ascending=False)
        .groupby("transcript_id")
        .head(1),
    ],
    axis=0,
    ignore_index=True,
).sort_values(["Coding_prob"], ascending=False).sort_values(
    "Coding_prob", ascending=False
).groupby(
    "transcript_id"
)

In [13]:
temp.head()

,ID,mRNA,ORF_strand,ORF_frame,ORF_start,ORF_end,ORF,Fickett,Hexamer,Coding_prob,transcript_id
0,TRANSCRIPT_162592_ORF_1,6719,+,2,197,6100,5904,1.0112,0.015127,1.0,TRANSCRIPT
1,TRANSCRIPT_162592_ORF_1,6719,+,2,197,6100,5904,1.0112,0.015127,1.0,TRANSCRIPT
2,TRANSCRIPT_162592_ORF_1,6719,+,2,197,6100,5904,1.0112,0.015127,1.0,TRANSCRIPT
